In [3]:
# earth engine things
import ee
import geemap
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Ab_5qlk9yukiTwanpG6H5WPlWF-qyTOfOCbu1EoItDnDmENTwVNCOLpkV30



Successfully saved authorization token.


## Province Level

In [10]:
import pandas as pd

# load Hansen dataset
dataset = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

# bands
tree_cover = dataset.select('treecover2000')
loss_year = dataset.select('lossyear')

# define forest threshold
forest_2000 = tree_cover.gte(30)

# load Zambia province boundaries
# level 0: country level
# level 1: province level
provinces = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# create empty list to store data
results = []

for year in range(2001, 2023):
    year_offset = year - 2000
    loss_in_year = loss_year.eq(year_offset)
    # mask to forested pixels in loss year
    loss_year_forest = loss_in_year.And(forest_2000)
    # convert to area (square meters)
    loss_area = loss_year_forest.multiply(ee.Image.pixelArea())
    # get total loss area for each province
    def compute_loss(feature):
        # province names
        name = feature.get('ADM1_NAME')
        stats = loss_area.reduceRegion(
            reducer = ee.Reducer.sum(),
            geometry = feature.geometry(),
            scale = 30,
            maxPixels = 1e13
        )
       
        area_m2 = stats.get('lossyear')
        return feature.set({
            'province': name, 
            'year': year,
            'loss_m2': area_m2
        })
        
    yearly_results = provinces.map(compute_loss).getInfo()

    # extract data to Python
    for feat in yearly_results['features']:
        properties = feat['properties']
        area_m2 = properties['loss_m2'] or 0
        # convert to hectares
        area_ha = area_m2 / 10000
        results.append({
            'province': properties['province'],
            'year': properties['year'],
            'loss_ha': area_ha
        })

# convert to pandas dataframe
df = pd.DataFrame(results)

# save as csv
df.to_csv('province_lcluc.csv', index=False)     

## District Level

In [11]:
# level 2: district level
districts = ee.FeatureCollection('FAO/GAUL/2015/level2') \
    .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# create empty list to store data
results = []

for year in range(2001, 2023):
    year_offset = year - 2000
    loss_in_year = loss_year.eq(year_offset)
    # mask to forested pixels in loss year
    loss_year_forest = loss_in_year.And(forest_2000)
    # convert to area (square meters)
    loss_area = loss_year_forest.multiply(ee.Image.pixelArea())
    # get total loss area for each province
    def compute_loss(feature):
        # district names
        name = feature.get('ADM2_NAME')
        stats = loss_area.reduceRegion(
            reducer = ee.Reducer.sum(),
            geometry = feature.geometry(),
            scale = 30,
            maxPixels = 1e13
        )
       
        area_m2 = stats.get('lossyear')
        return feature.set({
            'district': name, 
            'year': year,
            'loss_m2': area_m2
        })
        
    yearly_results = districts.map(compute_loss).getInfo()

    # extract data to Python
    for feat in yearly_results['features']:
        properties = feat['properties']
        area_m2 = properties['loss_m2'] or 0
        # convert to hectares
        area_ha = area_m2 / 10000
        results.append({
            'district': properties['district'],
            'year': properties['year'],
            'loss_ha': area_ha
        })

# convert to pandas dataframe
df = pd.DataFrame(results)

# save as csv
df.to_csv('district_lcluc.csv', index=False)

print(df.tail())

        district  year      loss_ha
1579     Zambezi  2022  3904.320786
1580      Kalabo  2022    54.157198
1581      Lukulu  2022  2196.119963
1582     Senanga  2022   116.785268
1583  Shang'ombo  2022   216.436116


## Code graveyard 

In [ ]:
#year = 2018

#loss_in_year = loss_year.eq(year - 2000)

# mask to forested pixels in loss year
#loss_year_forest = loss_in_year.And(forest_2000)

# convert to area (square meters)
#loss_area = loss_year_forest.multiply(ee.Image.pixelArea())

# load Zambia boundary
#zambia = ee.FeatureCollection('FAO/GAUL/2015/level0') \
            #.filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# get total loss area
#stats = loss_area.reduceRegion(
 #   reducer = ee.Reducer.sum(),
  #  geometry = zambia,
   # scale = 30,
    #maxPixels = 1e13
#)

# get result in hectares
#area_m2 = stats.getInfo().get('lossyear', 0)
#area_ha = area_m2 / 10000

#print(f"Forest loss in {year}: {area_ha:.2f} hectares")

In [ ]:
# create interactive map
Map = geemap.Map(center=[-13.1, 27.8], zoom=6)
    

# clip dataset to Zambia
#clipped_dataset = dataset.clip(zambia)

# visualization parameters
tree_cover_vis = {
    'bands': ['treecover2000'],
    'min': 0,
    'max': 100,
    'palette': ['black', 'green']
}

loss_vis = {
    'min': 0,
    'max': 23,
    'palette': ['yellow', 'red']
}

# add base tree cover layer
Map.addLayer(clipped_dataset.select('treecover2000'), tree_cover_vis, 'Tree Cover 2000')

# loop through 'test years'
for year in range(2001, 2021, 5):
    code = year - 2000
    annual_loss = clipped_dataset.select('lossyear') \
                        .eq(code) \
                        .selfMask()
    Map.addLayer(annual_loss, {'palette': ['red']}, f'Tree Loss {year}')

Map